In [17]:
from dotenv import load_dotenv
import urllib.request
import urllib.parse
import os
import json
import pandas as pd
from bs4 import BeautifulSoup
load_dotenv("C:/wanted/Lang/Presentation-Agent/.env")

True

In [18]:
print("현재 실행 중인 디렉토리:", os.getcwd())

현재 실행 중인 디렉토리: c:\wanted\Lang\Presentation-Agent\code


In [ ]:
def naver_developer(search, site, max_results=500):
    """네이버 API를 사용하여 데이터를 검색하고 'title'과 'link'만 Pandas DataFrame으로 반환"""
    
    encText = urllib.parse.quote(search)
    client_id = os.getenv("NAVER_CLIENT_ID")
    client_secret = os.getenv("NAVER_CLIENT_SECRET")
    
    if not client_id or not client_secret:
        raise ValueError("NAVER_CLIENT_ID 또는 NAVER_CLIENT_SECRET이 설정되지 않았습니다.")

    all_results = []  # 전체 데이터 저장 리스트
    display = 100  # 한 번 요청에 가져올 최대 개수 (최대 100)
    
    for start in range(1, max_results + 1, display):  # 1부터 max_results까지 100개씩 요청
        url = f"https://openapi.naver.com/v1/search/{site}.json?query={encText}&display={display}&start={start}&sort=date"
        print(f"Fetching: {url}")  # 요청 URL 출력

        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)

        try:
            response = urllib.request.urlopen(request)
            rescode = response.getcode()

            if rescode == 200:
                response_body = response.read()
                response_data = response_body.decode('utf-8')
                response_dict = json.loads(response_data)
                
                items = response_dict.get("items", [])
                
                # title과 link만 저장
                filtered_items = [{"title": item["title"], "link": item["link"]} for item in items]
                all_results.extend(filtered_items)  

                if len(items) < display:  # 마지막 페이지면 종료
                    break
            else:
                print(f"Error Code: {rescode}")
                break

        except urllib.error.HTTPError as e:
            print(f"HTTP Error: {e.code} - {e.reason}")
            break
        except urllib.error.URLError as e:
            print(f"URL Error: {e.reason}")
            break

    # Pandas DataFrame으로 변환
    csv_data = pd.DataFrame(all_results)

    return csv_data

def parsing(df, site):
    """ `BeautifulSoup`을 사용하여 HTML 태그 제거 후 기존 데이터프레임에 반영 """
    
    def clean_html(text):
        """HTML 태그 제거"""
        if pd.isna(text):  # NaN 값이 있는 경우 빈 문자열로 대체
            return ""
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text().strip()  # HTML 태그 제거 후 공백 제거
    
    # 'title'에서 HTML 태그 제거 (link는 HTML 태그 없음)
    df['title'] = df['title'].apply(clean_html)
    
    # CSV 파일로 저장
    csv_file_name = f"naver_{site}.csv"
    save_path = f"../data/csv/{csv_file_name}"
    df.to_csv(save_path, index=False, encoding="utf-8-sig")

    print(f" 데이터가 정제되어 저장되었습니다: {save_path}")
    
    return df 

In [30]:
search_list = ['webkr', 'doc', 'kin', "cafearticle"]

site_dict = {}
for site in search_list:
    df = naver_developer("발표 준비", site)
    df_parsing = parsing(df,site) 

    site_dict[site] = pd.read_csv(f"../data/csv/naver_{site}.csv")

Fetching: https://openapi.naver.com/v1/search/webkr.json?query=%EB%B0%9C%ED%91%9C%20%EC%A4%80%EB%B9%84&display=100&start=1&sort=date
데이터가 정제되어 저장되었습니다: ../data/csv/naver_webkr.csv
Fetching: https://openapi.naver.com/v1/search/doc.json?query=%EB%B0%9C%ED%91%9C%20%EC%A4%80%EB%B9%84&display=100&start=1&sort=date
Fetching: https://openapi.naver.com/v1/search/doc.json?query=%EB%B0%9C%ED%91%9C%20%EC%A4%80%EB%B9%84&display=100&start=101&sort=date
Fetching: https://openapi.naver.com/v1/search/doc.json?query=%EB%B0%9C%ED%91%9C%20%EC%A4%80%EB%B9%84&display=100&start=201&sort=date
Fetching: https://openapi.naver.com/v1/search/doc.json?query=%EB%B0%9C%ED%91%9C%20%EC%A4%80%EB%B9%84&display=100&start=301&sort=date
Fetching: https://openapi.naver.com/v1/search/doc.json?query=%EB%B0%9C%ED%91%9C%20%EC%A4%80%EB%B9%84&display=100&start=401&sort=date
Fetching: https://openapi.naver.com/v1/search/doc.json?query=%EB%B0%9C%ED%91%9C%20%EC%A4%80%EB%B9%84&display=100&start=501&sort=date
Fetching: https://openap

In [33]:
site_dict['webkr']

,title,link,description
0,발표수업에서 발표준비는 어떻게 해야 할까?(아주 길어요.하지만 발표준비의 정석입니다.),https://blog.naver.com/gritspeech/221934860734,GRITSPEECH 발표준비방법 발표수업에서 발표준비는 어떻게 해야 할까? 안녕하세...
1,중요한 발표 잘하는 요령 6,https://miraclespeaking.com/%EC%A4%91%EC%9A%94...,"중요한 발표를 잘 하기 위한 요령은 체계적인 준비, 자신감, 효과적인 전달 방식, ..."
2,학부연구생 논문 리뷰 발표 준비 못했는데 괜찮을까요.. - 김박사넷 커뮤니티,https://phdkim.net/board/free/57506,학부연구생으로 자대 신생랩에서 저와 동기 2명이서 교수님 지도 하에 논문 리뷰 랩세...
3,누구나 잘할 수 있는 발표 노하우 준비부터 실전까지,https://kmong.com/gig/558919,"발표하는직장인 전문가의 전자책 서비스를 만나보세요. "" 갑자기 발표 하래. 발표 준..."
4,"파워포인트 발표 준비를 위한 7가지 필수 팁 | 효과적인 발표, 프레젠테이션 기술,...",https://google.dkevent.kr/%ED%8C%8C%EC%9B%8C%E...,"파워포인트 발표 준비를 위한 7가지 필수 팁 | 효과적인 발표, 프레젠테이션 기술,..."
...,...,...,...
70,발표준비#KR1 - 게임 전적,https://www.op.gg/summoners/kr/%EB%B0%9C%ED%91...,플레이한 챔피언 (최근 3게임) ; 100% (1승 0패)2.57 평점 ; 0% (...
71,"2025 인천 고등학교 배정 발표: 일정, 기준, 준비 사항",https://thatstoryismine.tistory.com/325,목차 1. 2025 인천 고등학교 배정 일정 2. 인천 고등학교 배정 기준 및 지역...
72,대전 중학교 배정 발표: 2025학년도 대전 중학교 배정 일정과 준비 가이드,https://thatstoryismine.tistory.com/327,목차 1. 2025 대전 중학교 배정 발표 일정 2. 대전 중학교 배정 기준 및 지...
73,"""바드 발표 준비 부실했다""…구글서 커지는 피차이 CEO 비판",https://www.hankyung.com/article/202302121036i,"바드 발표 준비 부실했다…구글서 커지는 피차이 CEO 비판, 구글 내부서 시연 준비..."
